In [1]:
!pip install chromadb langchain pypdf2 tiktoken streamlit python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [6]:
import os

#from dotenv import load_dotenv

#import streamlit as st

#from langchain.document_loaders import PyPDFLoader
#from langchain.text_splitter import CharacterTextSplitter
#from langchain.embeddings import OpenAIEmbeddings
#from langchain.chains.question_answering import load_qa_chain
#from langchain.chat_models import ChatOpenAI
#from langchain.vectorstores import Chroma
import chromadb

ModuleNotFoundError: No module named 'langchain'

In [ ]:
#HUGGINGFACEHUB_API_TOKEN="<YOUR_KEY>"
#PINECONE_API_KEY="<YOUR_KEY>"
#PINECONE_ENV=environment="<YOUR_KEY>"
#SENTENCE_TRANSFORMERS_HOME="D:\\testing_space\\sentence_transformers_home"
#OPENAI_API_KEY="<YOUR_KEY>"

In [ ]:
def load_chunk_persist_pdf() -> Chroma:
    pdf_folder_path = "../book_sample/"
    documents = []
    for file in os.listdir(pdf_folder_path):
        if file.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder_path, file)
            loader = PyPDFLoader(pdf_path)
            documents.extend(loader.load())
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
    chunked_documents = text_splitter.split_documents(documents)
    client = chromadb.Client()
    if client.list_collections():
        consent_collection = client.create_collection("book_collection")
    else:
        print("Collection already exists")
    vectordb = Chroma.from_documents(
        documents=chunked_documents,
        embedding=OpenAIEmbeddings(),
        persist_directory="./chroma_store/"
    )
    vectordb.persist()
    return vectordb

In [ ]:
def create_agent_chain():
    model_name = "gpt-3.5-turbo"
    llm = ChatOpenAI(model_name=model_name)
    chain = load_qa_chain(llm, chain_type="stuff")
    return chain

In [ ]:
def get_llm_response(query):
    vectordb = load_chunk_persist_pdf()
    chain = create_agent_chain()
    matching_docs = vectordb.similarity_search(query)
    answer = chain.run(input_documents=matching_docs, question=query)
    return answer


# Streamlit UI
# ===============
st.set_page_config(page_title="Doc Searcher", page_icon=":robot:")
st.header("Query PDF Source")

form_input = st.text_input('Enter Query')
submit = st.button("Generate")

if submit:
    st.write(get_llm_response(form_input))

In [ ]:
streamlit run <FILE_NAME>.py